# Attribute Evaluation

https://www.goal.com/en-us/news/fifa-player-ratings-explained-how-are-the-card-number--stats-decided/1hszd2fgr7wgf1n2b2yjdpgynu

In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [102]:
df = pd.read_csv('dataset.csv')
df = df[df['Position'] != 'Goalkeeper']
df = df .drop('index', axis = 1)
df = df.drop('level_0', axis = 1)
df.reset_index(drop = True, inplace = True)
df.head(5)

,Player,Age,Matches Played,Start,Minutes,Goals,Assists,Penalties Made,Yellow Cards,Red Cards,...,Successful Challenge %,Blocks,Interceptions,Tackles + Interceptions,Clearances,Errors,Club,Position,Nation,Value
0,Bruno Fernandes,27,37,37,"3,316",8.0,8.0,2.0,6.0,0.0,...,42.3,37.0,25,93.0,40.0,1.0,Manchester United,midfield,Portugal,74900000.0
1,Randal Kolo Muani,23,32,31,"2,631",15.0,11.0,2.0,6.0,1.0,...,11.1,19.0,3,17.0,13.0,0.0,Eintracht Frankfurt,Attack,France,64000000.0
2,Victor Osimhen,23,32,30,"2,566",26.0,4.0,2.0,4.0,0.0,...,0.0,6.0,6,16.0,29.0,0.0,SSC Napoli,Attack,Nigeria,55000000.0
3,Bukayo Saka,20,38,37,"3,181",14.0,11.0,2.0,6.0,0.0,...,45.5,38.0,14,77.0,19.0,0.0,Arsenal FC,Attack,England,55000000.0
4,Vinicius Junior,22,33,32,"2,823",10.0,9.0,0.0,10.0,1.0,...,35.9,24.0,6,36.0,1.0,0.0,Real Madrid,Attack,Brazil,50000000.0


In [109]:
df2 = pd.read_csv('input_dataset.csv')
df2 = df2[df2['Position'] != 'Goalkeeper']
df2 = df2 .drop('index', axis = 1)
df2 = df2.drop('level_0', axis = 1)
df2.reset_index(drop = True, inplace = True)
df2.head(5)

,Player,Age,Matches Played,Start,Minutes,Goals,Assists,Penalties Made,Yellow Cards,Red Cards,...,Successful Challenge %,Blocks,Interceptions,Tackles + Interceptions,Clearances,Errors,Club,Position,Nation,Value
0,Sadio Mane,30,25,18,"1,425",7.0,5.0,1.0,2.0,0.0,...,11.1,15.0,0,11.0,4.0,0.0,Bayern Munich,Attack,Senegal,-45000000.0
1,Fernández Enzo,21,18,18,"1,547",0.0,2.0,0.0,3.0,0.0,...,37.5,32.0,16,61.0,12.0,1.0,Chelsea FC,midfield,Argentina,45000000.0
2,Mac Allister Alexis,23,35,31,"2,886",10.0,2.0,6.0,8.0,0.0,...,48.6,26.0,22,92.0,10.0,1.0,Liverpool FC,midfield,Argentina,45000000.0


In [104]:
selected_columns = ['Goals',
       'Assists', 'Penalties Made', 'Yellow Cards', 'Red Cards', 'Shots',
       'Shots On Target', 'Shots On Target %', 'Goals Per Shot',
       'Goals Per Shot On Target', 'Average Shot Distance',
       'Shots From Freekicks', 'Passes Completed', 'Passes',
       'Pass Completion %', 'Total Passing Distance',
       'Progressive Passing Distance', 'Shot-Creating Actions',
       'Shot-Creating Actions/90', 'SCA Pass Live', 'SCA Pass Dead',
       'SCA Take-ons', 'SCA Shots', 'SCA Fouls', 'SCA Defense',
       'Goal-Creating Actions', 'Goal-Creating Actions/90', 'Goal Pass Live',
       'Goal Pass Dead', 'Goal Take-ons', 'Goal Shots', 'Goal Fouls',
       'Goal Defense', 'Tackles', 'Tackles Won', 'Challenges',
       'Successful Challenge %', 'Blocks', 'Interceptions',
       'Tackles + Interceptions', 'Clearances', 'Errors']
selected = df[selected_columns]
scaler = MinMaxScaler(feature_range=(0, 99))
selected_scaled = pd.DataFrame(scaler.fit_transform(selected), columns=selected_columns)
df[selected_columns] = selected_scaled

In [105]:
selected = df2[selected_columns]
selected_scaled = pd.DataFrame(scaler.transform(selected), columns=selected_columns)
df2[selected_columns] = selected_scaled

In [106]:
df['Shooting'] = 0.30*df['Shots On Target'] / df['Matches Played'] + 0.45*df['Goals'] / df['Matches Played'] + 0.15*df['Goals Per Shot'] + 0.05*df['Penalties Made'] / df['Matches Played'] + 0.05*df['Shots From Freekicks'] / df['Matches Played']
df['Passing'] = 0.45*df['Assists'] / df['Matches Played'] + 0.05*df['Passes Completed'] / df['Matches Played'] + 0.025*df['Progressive Passing Distance'] / df['Matches Played'] + 0.025*df['Total Passing Distance'] / df['Matches Played'] + 0.45*df['Shot-Creating Actions/90']
df['Dribbling'] = 0.40*df['SCA Take-ons'] / df['Matches Played'] + 0.40*df['Goal Take-ons'] / df['Matches Played'] + 0.10*df['Shot-Creating Actions/90'] + 0.10*df['Goal-Creating Actions/90']
df['Defending'] = 0.20*df['Successful Challenge %']*df['Challenges'] / df['Matches Played'] + 0.20*df['Tackles Won'] / df['Matches Played'] + 0.20*df['Interceptions'] / df['Matches Played'] + 0.15*df['Clearances'] / df['Matches Played'] + 0.20*df['Blocks'] / df['Matches Played'] - 0.05*df['Errors'] / df['Matches Played']
df["Physical"] = 0.20*df['Yellow Cards'] / df['Matches Played'] + 0.20*df['Red Cards'] / df['Matches Played'] + 0.20*df['Tackles'] / df['Matches Played'] + 0.20*df['Challenges'] / df['Matches Played'] + 0.10*df['SCA Defense'] / df['Matches Played'] + 0.10*df['Goal Defense'] / df['Matches Played']

In [77]:
df2['Shooting'] = 0.30*df2['Shots On Target'] / df2['Matches Played'] + 0.45*df2['Goals'] / df2['Matches Played'] + 0.15*df2['Goals Per Shot'] + 0.05*df2['Penalties Made'] / df2['Matches Played'] + 0.05*df2['Shots From Freekicks'] / df2['Matches Played']
df2['Passing'] = 0.45*df2['Assists'] / df2['Matches Played'] + 0.05*df2['Passes Completed'] / df2['Matches Played'] + 0.025*df2['Progressive Passing Distance'] / df2['Matches Played'] + 0.025*df2['Total Passing Distance'] / df2['Matches Played'] + 0.45*df2['Shot-Creating Actions/90']
df2['Dribbling'] = 0.40*df2['SCA Take-ons'] / df2['Matches Played'] + 0.40*df2['Goal Take-ons'] / df2['Matches Played'] + 0.10*df2['Shot-Creating Actions/90'] + 0.10*df2['Goal-Creating Actions/90']
df2['Defending'] = 0.20*df2['Successful Challenge %']*df2['Challenges'] / df2['Matches Played'] + 0.20*df2['Tackles Won'] / df2['Matches Played'] + 0.20*df2['Interceptions'] / df2['Matches Played'] + 0.15*df2['Clearances'] / df2['Matches Played'] + 0.20*df2['Blocks'] / df2['Matches Played'] - 0.05*df2['Errors'] / df2['Matches Played']
df2["Physical"] = 0.20*df2['Yellow Cards'] / df2['Matches Played'] + 0.20*df2['Red Cards'] / df2['Matches Played'] + 0.20*df2['Tackles'] / df2['Matches Played'] + 0.20*df2['Challenges'] / df2['Matches Played'] + 0.10*df2['SCA Defense'] / df2['Matches Played'] + 0.10*df2['Goal Defense'] / df2['Matches Played']

In [78]:
df['Shooting'] = 0.40*df['Matches Played'] + 0.60*df['Shooting']
df['Passing'] = 0.40*df['Matches Played'] + 0.60*df['Passing']
df['Dribbling'] = 0.40*df['Matches Played'] + 0.60*df['Dribbling']
df['Defending'] = 0.40*df['Matches Played'] + 0.60*df['Defending']
df["Physical"] = 0.40*df['Matches Played'] + 0.60*df['Physical']

In [79]:
df2['Shooting'] = 0.40*df2['Matches Played'] + 0.60*df2['Shooting']
df2['Passing'] = 0.40*df2['Matches Played'] + 0.60*df2['Passing']
df2['Dribbling'] = 0.40*df2['Matches Played'] + 0.60*df2['Dribbling']
df2['Defending'] = 0.40*df2['Matches Played'] + 0.60*df2['Defending']
df2["Physical"] = 0.40*df2['Matches Played'] + 0.60*df2['Physical']

In [80]:
selected_columns = ['Player', 'Age', 'Position', 'Club', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating = df[selected_columns].copy()
rating.replace([np.inf, -np.inf], 0, inplace=True)
rating.tail()

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical
1600,Carlos Soler,25,midfield,Paris Saint-Germain,13.215006,15.155480,11.743760,13.770759,10.523913
1601,Ansu Fati,19,Attack,FC Barcelona,15.658316,20.309955,16.206521,15.141690,14.561930
1602,Romelu Lukaku,29,Attack,Chelsea FC,11.941648,17.452682,12.367703,10.555241,10.056484
1603,Jadon Sancho,22,Attack,Manchester United,13.239746,18.891688,12.887364,12.021274,10.499130
1604,Sadio Mane,30,Attack,Bayern Munich,11.395168,17.197732,12.165491,10.177824,10.109867


In [81]:
selected_columns = ['Player', 'Age', 'Position', 'Club', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating2 = df2[selected_columns].copy()
rating2.replace([np.inf, -np.inf], 0, inplace=True)
rating2.tail()

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical
0,Sadio Mane,30,Attack,Bayern Munich,11.395168,17.197732,12.165491,10.177824,10.109867
1,Fernández Enzo,21,midfield,Chelsea FC,7.256399,13.637108,8.743080,14.716872,7.730767
2,Mac Allister Alexis,23,midfield,Liverpool FC,14.936171,20.073437,15.766025,21.846623,14.579431


In [82]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
selected = df[selected_columns]
scaler = MinMaxScaler(feature_range=(0, 99))
selected_scaled = pd.DataFrame(scaler.fit_transform(selected), columns=selected_columns)
rating[selected_columns] = selected_scaled

In [83]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
selected = rating2[selected_columns]
selected_scaled = pd.DataFrame(scaler.transform(selected), columns=selected_columns)
rating2[selected_columns] = selected_scaled

In [84]:
rating

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical
0,Bruno Fernandes,27,midfield,Manchester United,86.989626,91.171735,96.903033,60.923267,94.777124
1,Randal Kolo Muani,23,Attack,Eintracht Frankfurt,81.780802,69.467210,85.583627,38.536189,81.388143
2,Victor Osimhen,23,Attack,SSC Napoli,84.692298,63.153320,77.828189,38.284471,80.635528
3,Bukayo Saka,20,Attack,Arsenal FC,93.145659,84.074420,97.948382,57.021780,97.577449
4,Vinicius Junior,22,Attack,Real Madrid,81.439475,80.432157,91.919860,46.789164,85.715518
...,...,...,...,...,...,...,...,...,...
1600,Carlos Soler,25,midfield,Paris Saint-Germain,72.226724,52.944508,63.905008,40.778732,64.760786
1601,Ansu Fati,19,Attack,FC Barcelona,85.997473,71.439406,89.045948,44.782011,90.591231
1602,Romelu Lukaku,29,Attack,Chelsea FC,65.049942,61.187154,67.419986,31.389042,61.770732
1603,Jadon Sancho,22,Attack,Manchester United,72.366159,66.350488,70.347492,35.670033,64.602257


In [85]:
rating2 = rating2.dropna()
rating2

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical
0,Sadio Mane,30,Attack,Bayern Munich,61.969923,60.272363,66.280829,30.286943,62.112210
1,Fernández Enzo,21,midfield,Chelsea FC,38.643388,47.496399,47.000694,43.541494,46.893551
2,Mac Allister Alexis,23,midfield,Liverpool FC,81.927393,70.590753,86.564416,64.361202,90.703183


In [86]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating['Overall'] = rating[selected_columns].apply(lambda x: x.nlargest(4).mean(), axis=1)

In [87]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating2['Overall'] = rating2[selected_columns].apply(lambda x: x.nlargest(4).mean(), axis=1)

In [88]:
rating.sort_values(by = 'Defending', ascending=False).head(20)

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical,Overall
1081,Andrei Girotto,30,Defender,FC Nantes,81.304015,56.531065,81.315435,99.000000,94.684857,89.076077
1189,Melvin Bard,21,Defender,OGC Nice,72.225532,53.220119,74.712904,96.248637,86.206118,82.348298
73,Joao Palhinha,27,midfield,Fulham FC,81.380930,54.570570,78.556374,93.537967,94.290934,86.941552
488,Danilo,31,Defender,Juventus FC,86.226284,63.729277,85.353523,92.208527,95.576365,89.841175
405,Reinildo Mandava,28,Defender,Atlético de Madrid,47.343285,37.597111,49.968644,90.673315,58.390322,61.593891
1163,Nicolas Otamendi,34,Defender,SL Benfica,69.674222,50.047615,70.033573,90.526052,82.702741,78.234147
1293,Benjamin Andre,31,midfield,LOSC Lille,76.690243,61.773127,79.745943,90.204327,90.710299,84.337703
769,Juan Foyth,24,Defender,Villarreal CF,55.771079,41.864854,55.062900,87.637254,63.534537,65.501443
846,Bryan Cristante,27,midfield,AS Roma,81.221514,58.932488,82.371377,87.448373,94.656951,86.424554
838,Vincent Le Goff,32,Defender,FC Lorient,81.383024,58.599697,83.702818,86.209051,94.763114,86.514502


In [89]:
rating['Value'] = df['Value']

In [90]:
rating2['Value'] = df2['Value']

In [91]:
rating2.sort_values(by = 'Passing', ascending=False).head(20)

,Player,Age,Position,Club,Shooting,Passing,Dribbling,Defending,Physical,Overall,Value
2,Mac Allister Alexis,23,midfield,Liverpool FC,81.927393,70.590753,86.564416,64.361202,90.703183,82.446436,45000000.0
0,Sadio Mane,30,Attack,Bayern Munich,61.969923,60.272363,66.280829,30.286943,62.112210,62.658831,-45000000.0
1,Fernández Enzo,21,midfield,Chelsea FC,38.643388,47.496399,47.000694,43.541494,46.893551,46.233035,45000000.0


# Scaling Defenders Stats

In [92]:
for i in range(len(rating['Position'])):
    if rating.loc[i, 'Position'] == 'Defender':
        rating.loc[i, 'Shooting'] = 0.80 * rating.loc[i, 'Shooting']
        rating.loc[i, 'Dribbling'] = 0.90 * rating.loc[i, 'Dribbling']
        rating.loc[i, 'Passing'] = 1.05 * rating.loc[i, 'Passing']

def normalize_physical_stat(value):
    if value < 75 and value > 50:
        return value + 20
    if value < 50:
        return value + 45
    else:
        return value

rating.loc[(rating['Position'] == 'Defender'), 'Physical'] = rating.loc[(rating['Position'] == 'Defender'), 'Physical'].apply(normalize_physical_stat)

In [93]:
for i in range(len(rating['Position'])):
    if rating.loc[i, 'Position'] == 'Defender':
        rating.loc[i, 'Shooting'] = 0.80 * rating.loc[i, 'Shooting']
        rating.loc[i, 'Dribbling'] = 0.90 * rating.loc[i, 'Dribbling']
        rating.loc[i, 'Passing'] = 1.05 * rating.loc[i, 'Passing']

def normalize_physical_stat(value):
    if value < 75 and value > 50:
        return value + 20
    if value < 50:
        return value + 45
    else:
        return value

rating2.loc[(rating2['Position'] == 'Defender'), 'Physical'] = rating2.loc[(rating2['Position'] == 'Defender'), 'Physical'].apply(normalize_physical_stat)

## Scaling Midfielders Stats

In [94]:
for i in range(len(rating['Position'])):
    if rating.loc[i, 'Position'] == 'midfield':
        rating.loc[i, 'Passing'] = 1.03 * rating.loc[i, 'Passing']

In [95]:
for i in range(len(rating2['Position'])):
    if rating2.loc[i, 'Position'] == 'midfield':
        rating2.loc[i, 'Passing'] = 1.03 * rating2.loc[i, 'Passing']

# Assigning teams into leagues

In [96]:
England =  [
        'Manchester United', 'Arsenal FC', 'Newcastle United', 'Everton FC',
        'Manchester City', 'Chelsea FC', 'Nottingham Forest', 'Liverpool FC',
        'AFC Bournemouth', 'West Ham United', 'Crystal Palace', 'Wolverhampton Wanderers',
        'Tottenham Hotspur', 'Aston Villa', 'Fulham FC', 'Brentford FC', 'Burnley FC'
    ]

Spain = [
        'Real Madrid', 'FC Barcelona', 'Athletic Bilbao', 'Villarreal CF',
        'Atlético de Madrid', 'Celta de Vigo', 'RCD Mallorca', 'Real Betis Balompié',
        'Valencia CF', 'CA Osasuna', 'Deportivo Alavés', 'Sevilla FC', 'Getafe CF',
        'Cádiz CF', 'UD Las Palmas', 'Real Sociedad', 'Girona FC', 'Rayo Vallecano'
    ]

Germany = [
        'Bayern Munich', 'RB Leipzig', 'Bayer 04 Leverkusen', 'Borussia Dortmund',
        'SC Freiburg', 'SV Werder Bremen', 'FC Augsburg', 'VfL Wolfsburg', 'VfB Stuttgart',
        'Borussia Mönchengladbach', '1.FSV Mainz 05', '1.FC Köln', 'VfL Bochum',
        '1.FC Heidenheim 1846', 'Eintracht Frankfurt', '1.FC Union Berlin', 'TSG 1899 Hoffenheim'
    ]

Italy =  [
        'SSC Napoli', 'Atalanta BC', 'Juventus FC', 'Torino FC', 'AC Milan',
        'ACF Fiorentina', 'Udinese Calcio', 'Inter Milan', 'AS Roma', 'Hellas Verona',
        'Genoa CFC', 'SS Lazio', 'FC Empoli', 'US Salernitana 1919', 'US Sassuolo', 'Bologna FC 1909',
        'US Lecce', 'AC Monza', 'Frosinone Calcio'
    ]

France =  [
        'Paris Saint-Germain', 'RC Lens', 'LOSC Lille', 'Stade Rennais FC',
        'OGC Nice', 'Montpellier HSC', 'AS Monaco', 'Olympique Lyon', 'Olympique Marseille',
        'Clermont Foot 63', 'Stade Reims', 'UD Almería', 'FC Toulouse', 'FC Nantes',
        'RC Strasbourg Alsace', 'FC Lorient', 'Stade Brestois 29', 'FC Metz'
    ]

Netherlands =  [
        'Feyenoord Rotterdam', 'AZ Alkmaar', 'Ajax Amsterdam', 'PSV Eindhoven',
        'Sparta Rotterdam', 'RKC Waalwijk', 'NEC Nijmegen', 'Go Ahead Eagles',
        'FC Utrecht', 'Vitesse Arnhem', 'Fortuna Sittard', 'Excelsior Rotterdam', 'FC Volendam',
        'SC Heerenveen', 'Twente Enschede FC'
    ]
Portugal =  [
        'SL Benfica', 'FC Porto', 'Sporting CP', 'FC Famalicão', 'Rio Ave FC',
        'Gil Vicente FC', 'GD Estoril Praia', 'FC Arouca', 'Portimonense SC', 'SC Braga',
        'Vitória Guimarães SC', 'SC Farense', 'Casa Pia AC', 'Boavista FC', 'FC Vizela',
        'GD Chaves', 'CF Estrela Amadora SAD'
    ]

Belgium = [
        'RWD Molenbeek', 'RSC Anderlecht', 'Royal Antwerp FC', 'KAS Eupen',
        'Cercle Brugge', 'Club Brugge KV', 'KAA Gent', 'Charleroi SC','Cercle Brugge',
        'RSC Charleroi', 'Cercle Brugge '
    ]

Turkey = [
        'Fenerbahce', 'Kasimpasa', 'Galatasaray', 'Alanyaspor', 'Hatayspor',
        'MKE Ankaragücü'
    ]

In [97]:
LeagueList = []
for i in rating['Club']:
    if i in England:
        LeagueList.append('Premier League')
    elif i in Spain:
        LeagueList.append('La Liga')
    elif i in Italy:
        LeagueList.append('Serie A')
    elif i in Germany:
        LeagueList.append('Bundesliga')
    elif i in France:
        LeagueList.append('Ligue 1')
    elif i in Netherlands:
        LeagueList.append('Eredivisie')
    elif i in Portugal:
        LeagueList.append('Primeira Liga')
    elif i in Belgium:
        LeagueList.append('Jupiler Pro League')
    elif i in Turkey:
        LeagueList.append('Super Lig ')
    else:
        LeagueList.append("NA")
        
rating['League'] = LeagueList

In [98]:
LeagueList = []
for i in rating2['Club']:
    if i in England:
        LeagueList.append('Premier League')
    elif i in Spain:
        LeagueList.append('La Liga')
    elif i in Italy:
        LeagueList.append('Serie A')
    elif i in Germany:
        LeagueList.append('Bundesliga')
    elif i in France:
        LeagueList.append('Ligue 1')
    elif i in Netherlands:
        LeagueList.append('Eredivisie')
    elif i in Portugal:
        LeagueList.append('Primeira Liga')
    elif i in Belgium:
        LeagueList.append('Jupiler Pro League')
    elif i in Turkey:
        LeagueList.append('Super Lig ')
    else:
        LeagueList.append("NA")
        
rating2['League'] = LeagueList

# Scaling League Weights for Defense Rating with Hakeem Number

In [99]:
selected = rating[(rating['Position'] == 'Defender') & (rating['League'] == 'Premier League')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value

def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Premier League'), 'Defending'] = selected.apply(normalize_defending_stat)
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Premier League'), 'Passing'] = selected.apply(normalize_passing_stat)


In [100]:
selected = rating2[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Premier League')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value

def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Premier League'), 'Defending'] = selected.apply(normalize_defending_stat)
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Premier League'), 'Passing'] = selected.apply(normalize_passing_stat)


ValueError: min() arg is an empty sequence

In [ ]:
selected = rating[(rating['Position'] == 'Defender') & (rating['League'] == 'La Liga')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value
    
def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'La Liga'), 'Defending'] = selected.apply(normalize_defending_stat)
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'La Liga'), 'Passing'] = selected.apply(normalize_passing_stat)


In [ ]:
selected = rating2[(rating2['Position'] == 'Defender') & (rating2['League'] == 'La Liga')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value
    
def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'La Liga'), 'Defending'] = selected.apply(normalize_defending_stat)
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'La Liga'), 'Passing'] = selected.apply(normalize_passing_stat)


In [ ]:
selected = rating[(rating['Position'] == 'Defender') & (rating['League'] == 'Bundesliga')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 27.5
    if value < 50:
        return value + 48
    else:
        return value
    
def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Bundesliga'), 'Defending'] = selected.apply(normalize_defending_stat)
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Bundesliga'), 'Passing'] = selected.apply(normalize_passing_stat)


In [ ]:
selected = rating2[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Bundesliga')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 27.5
    if value < 50:
        return value + 48
    else:
        return value
    
def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Bundesliga'), 'Defending'] = selected.apply(normalize_defending_stat)
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Bundesliga'), 'Passing'] = selected.apply(normalize_passing_stat)


In [ ]:
selected = rating[(rating['Position'] == 'Defender') & (rating['League'] == 'Serie A')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value

def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Serie A'), 'Defending'] = selected.apply(normalize_defending_stat)
rating.loc[(rating['Position'] == 'Defender') & (rating['League'] == 'Serie A'), 'Passing'] = selected.apply(normalize_passing_stat)


In [ ]:
selected = rating2[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Serie A')]['Defending']
min_defending_stat = min(selected)
max_defending_stat = max(selected)

def normalize_defending_stat(value):
    if value < 70 and value > 50:
        return value + 20
    if value < 50:
        return value + 48
    else:
        return value

def normalize_passing_stat(value):
    if value < 70 and value > 50:
        return value + 10
    if value < 50:
        return value + 35
    else:
        return value

# Use .loc to assign the scaled defending stat to the DataFrame
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Serie A'), 'Defending'] = selected.apply(normalize_defending_stat)
rating2.loc[(rating2['Position'] == 'Defender') & (rating2['League'] == 'Serie A'), 'Passing'] = selected.apply(normalize_passing_stat)


## Re-Compute Overall

In [ ]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating['Overall'] = rating[selected_columns].apply(lambda x: x.nlargest(4).mean(), axis=1)

In [ ]:
selected_columns = ['Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating2['Overall'] = rating2[selected_columns].apply(lambda x: x.nlargest(4).mean(), axis=1)

In [ ]:
rating = rating[['Player', 'Age', 'League', 'Club', 'Position', 'Value', 'Overall', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']]

In [ ]:
rating2 = rating2[['Player', 'Age', 'League', 'Club', 'Position', 'Value', 'Overall', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']]

In [ ]:
selected_columns = ['Overall', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating[selected_columns] = rating[selected_columns].round(2)

In [ ]:
selected_columns = ['Overall', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physical']
rating2[selected_columns] = rating2[selected_columns].round(2)

In [ ]:
rating.sort_values(by = 'Overall', ascending=False).head(20)

,Player,Age,League,Club,Position,Value,Overall,Shooting,Passing,Dribbling,Defending,Physical
1516,Antoine Griezmann,31,La Liga,Atlético de Madrid,Attack,-10000000.0,94.47,93.79,88.31,99.00,53.43,96.77
3,Bukayo Saka,20,Premier League,Arsenal FC,Attack,55000000.0,93.19,93.15,84.07,97.95,57.02,97.58
0,Bruno Fernandes,27,Premier League,Manchester United,midfield,74900000.0,93.14,86.99,93.91,96.90,60.92,94.78
55,Eberechi Eze,24,Premier League,Crystal Palace,midfield,18000000.0,92.20,92.29,82.02,97.95,56.03,96.56
836,Harry Kane,29,Premier League,Tottenham Hotspur,Attack,0.0,91.64,99.00,75.81,95.22,48.26,96.52
7,Martin Odegaard,23,Premier League,Arsenal FC,midfield,45000000.0,91.38,92.24,85.48,93.58,49.53,94.21
1599,Mohamed Salah,30,Premier League,Liverpool FC,Attack,-25000000.0,91.36,94.30,79.53,95.45,50.07,96.16
1335,Kieran Trippier,31,Premier League,Newcastle United,Defender,-4000000.0,91.36,87.44,85.21,95.00,75.31,97.80
1435,Dusan Tadic,33,Eredivisie,Ajax Amsterdam,Attack,-6000000.0,91.00,85.28,99.00,93.81,50.78,85.90
1398,Piotr Zielinski,28,Serie A,SSC Napoli,midfield,-5000000.0,90.77,85.35,89.69,94.90,51.47,93.12


In [ ]:
rating[rating['Player'] == 'Josko Gvardiol']

,Player,Age,League,Club,Position,Value,Overall,Shooting,Passing,Dribbling,Defending,Physical
10,Josko Gvardiol,20,Bundesliga,RB Leipzig,Defender,40000000.0,65.68,69.22,49.81,68.43,42.71,75.27


In [ ]:
rating2

,Player,Age,League,Club,Position,Value,Overall,Shooting,Passing,Dribbling,Defending,Physical
0,Sadio Mane,30,Bundesliga,Bayern Munich,Attack,-45000000.0,62.66,61.97,60.27,66.28,30.29,62.11
1,Fernández Enzo,21,NA,Chelsea FC,midfield,45000000.0,46.23,38.64,47.50,47.00,43.54,46.89
2,Mac Allister Alexis,23,NA,Liverpool FC,midfield,45000000.0,82.45,81.93,70.59,86.56,64.36,90.70


In [108]:
rating2.to_csv('input_Fifa_stats.csv')